## Preambule

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm
import xarray as xr

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

C:\Users\dekkerm\AppData\Local\Temp\2\ipykernel_776\2088128098.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Input

In [2]:
params = dict(start_year_analysis=2021)
version = '' # 'PBL/' or ''  
path_alloc = "K:/ECEMF/T5.2/Allocations/"+version
path_save = "K:/ECEMF/T5.2/"+version

## Read

In [9]:
xr_dataread = xr.open_dataset(path_save + "xr_dataread.nc", engine='netcdf4')
all_countries_iso = np.load(path_save + "all_countries.npy", allow_pickle=True)

In [4]:
focusregs = np.array(xr_dataread.Region)
if version == 'PBL/': temperatures = [1.56, 2.0]
else: temperatures = [1.5, 2.0]
rules = ['GF', 'PC', 'PCC', 'ECPC', 'AP', 'GDR']

## Save allocations by year

In [16]:
for year in np.arange(2035, 2101,5):#np.array([2030, 2040, 2050]):#, 2060, 2070, 2080, 2090, 2100]):
    print("Starting with", year)
    for cty_i, cty in tqdm(enumerate(focusregs)):
        ds = xr.open_dataset(path_alloc + "xr_alloc_"+cty+".nc").sel(Time=year).expand_dims(Region=[cty])
        if cty_i == 0:
            xrt = ds.copy()
        else:
            xrt = xr.merge([xrt, ds])
        ds.close()
    xrt.astype("float32").to_netcdf(path_save + "xr_alloc_"+str(year)+".nc", format='NETCDF4')

Starting with 2035


207it [22:17,  6.46s/it]


Starting with 2040


207it [22:11,  6.43s/it]


Starting with 2045


...

207it [20:21,  5.90s/it]


Starting with 2095


207it [20:19,  5.89s/it]


Starting with 2100


207it [20:45,  6.02s/it]


In [15]:
xr_dataread

<xarray.Dataset>
Dimensions:            (Scenario: 5, Region: 207, Time: 251, Conditionality: 3,
                        Hot_air: 2, Ambition: 2, Temperature: 10, Risk: 5,
                        NonCO2red: 5, NegEmis: 7, Timing: 2)
Coordinates: (12/14)
  * Scenario           (Scenario) <U4 'SSP1' 'SSP2' 'SSP3' 'SSP4' 'SSP5'
  * Region             (Region) <U16 'AFG' 'ALB' ... 'African Group' 'Umbrella'
  * Time               (Time) int32 1850 1851 1852 1853 ... 2097 2098 2099 2100
  * Conditionality     (Conditionality) <U13 'conditional' ... 'unconditional'
  * Hot_air            (Hot_air) <U7 'include' 'exclude'
  * Ambition           (Ambition) <U4 'low' 'high'
    ...                 ...
  * NonCO2red          (NonCO2red) float64 0.1 0.33 0.5 0.67 0.9
  * NegEmis            (NegEmis) float64 0.2 0.3 0.4 0.5 0.6 0.7 0.8
  * Timing             (Timing) <U9 'Delayed' 'Immediate'
    source             <U25 ...
    Version            <U6 ...
    variable           <U8 ...
Data variables: (12/15)
    GDP                (Scenario, Region, Time) float64 ...
    Population         (Scenario, Region, Time) float64 ...
    GHG_hist           (Time, Region) float64 ...
    CO2_hist           (Time, Region) float64 ...
    N2O_hist           (Time, Region) float64 ...
    CH4_hist           (Time, Region) float64 ...
    ...                 ...
    Budget             (Temperature, Risk, NonCO2red) float64 ...
    GHG_globe          (NegEmis, NonCO2red, Temperature, Risk, Timing, Time) float64 ...
    CO2_globe          (NegEmis, NonCO2red, Temperature, Risk, Timing, Time) float64 ...
    NonCO2_globe       (NonCO2red, Time, Timing, Temperature) float64 ...
    GHG_base           (Region, Scenario, Time) float64 ...
    GHG_ndc            (Region, Conditionality, Hot_air, Ambition, Time) float64 ...

Trajectory emissions

In [13]:
xrs = []
for cty in tqdm(all_countries_iso[:5]):
    xrs.append(xr.open_dataset(path_save + "Allocations/xr_alloc_"+cty+".nc").sel(Temperature=1.5, Risk=0.5, NonCO2red=0.5, Timing='Immediate', NegEmis=0.5, Time=np.arange(params['start_year_analysis'], 2101))['GF'].expand_dims(Region=[cty]))

100%|██████████| 5/5 [00:01<00:00,  3.53it/s]


In [ ]:
# for temp_i, temp in enumerate([1.5, 2.0]):
#     for cty_i, cty in tqdm(enumerate(focusregs)):
#         ds = xr.open_dataset(path_alloc + "xr_alloc_"+cty+".nc").sel(Temperature=temp, Risk=[0.5, 0.33][temp_i], Time=np.arange(params['start_year_analysis'], 2101)).expand_dims(Region=[cty])
#         if cty_i == 0:
#             xrt = ds.copy()
#         else:
#             xrt = xr.merge([xrt, ds])
#         ds.close()
#     xrt.astype("float32").to_netcdf(path_save + "xr_traj_"+str(temp)[0]+str(temp)[2]+"_50.nc", format='NETCDF4')